### Proceso de caratulas

Primero inicaremos tratando de replicar el proceso desde este notebook, iniciaremos cargando la base de sucursales:

In [127]:
import pandas as pd

In [253]:
from datetime import datetime

# Obtener la fecha actual
hoy = datetime.today()

# Truncar al primer día del mes
inicio_mes = hoy.replace(day=1)

# Formatear como 'YYYY-MM-DD'
fecha_truncada = inicio_mes.strftime('%Y-%m-%d')
mes_anio = inicio_mes.strftime('%Y%m')

print(mes_anio)
print(fecha_truncada)


202601
2026-01-01


In [129]:
inicio_mes2 = hoy.replace(month=9, day=1)
print(inicio_mes2)

2025-09-01 13:16:02.315547


In [130]:
fecha_truncada2 = inicio_mes2.strftime('%Y-%m-%d')
print(fecha_truncada2)

2025-09-01


### Cargaremos los archivos

In [131]:
import pandas as pd
import os
from openpyxl import load_workbook

# Carpeta con los archivos Excel
carpeta = "./VentasFinales/Diciembre/3112/"
#archivos_excel = [f for f in os.listdir(carpeta) if f.endswith('.xlsx')]
archivos_excel = [f for f in os.listdir(carpeta) if f.endswith('.xlsx') and not f.startswith('~$')]
dataframes = {}

for archivo in archivos_excel:
    ruta = os.path.join(carpeta, archivo)

    # Verificar si el archivo es un .xlsx válido
    try:
        _ = load_workbook(ruta)
    except Exception as e:
        raise ValueError(f"❌ Error al abrir el archivo '{archivo}': {e}")

    # Leer el archivo sin encabezado para buscar la fila inicial
    try:
        df_raw = pd.read_excel(ruta, header=None, engine="openpyxl")
    except Exception as e:
        raise ValueError(f"❌ Error leyendo '{archivo}' con pandas: {e}")

    # Buscar la fila que contiene "No." como punto de inicio
    inicio = None
    for i, row in df_raw.iterrows():
        if "No." in row.values:
            inicio = i
            break

    if inicio is not None:
        try:
            # Cargar la tabla desde la fila identificada
            df_tabla = pd.read_excel(ruta, header=inicio, engine="openpyxl")

            # Recortar al detectar la primera fila completamente vacía
            fila_nula = df_tabla.isnull().all(axis=1)
            if fila_nula.any():
                corte = fila_nula.idxmax()
                df_tabla = df_tabla.loc[:corte - 1]

            # Guardar el DataFrame usando el nombre del archivo sin extensión
            nombre = os.path.splitext(archivo)[0]
            dataframes[nombre] = df_tabla
        except Exception as e:
            raise ValueError(f"❌ Error procesando tabla en '{archivo}': {e}")
    else:
        print(f"⚠️ No se encontró encabezado en: {archivo}")


In [132]:
import time

In [133]:
for data in dataframes:
    print(dataframes[data], "El dataframe cargado es: ",data)

    No.  Contrato  Capital Insoluto  Dias Vencidos     DSM Mercado Producto
0   1.0  75248092          20736.49          499.0   367.0    INFO     POPN
1   2.0  78406574          32290.93         1624.0  1541.0    FORM     REVN
2   3.0  79058845           9487.58         1046.0   400.0    INFO     PFEN
3   4.0  79072473          26942.41          361.0   153.0    FORM     REVN
4   5.0  79115128           6256.73          849.0   644.0    INFO     PFEN
5   6.0  79145900           5808.31          650.0   400.0    INFO     PFEN
6   7.0  79171631           6098.92          605.0   400.0    INFO     PFEN
7   8.0  79183073          22106.59          562.0   400.0    INFO     PFEN
8   9.0  79208882           5362.03          532.0   400.0    INFO     PFEN
9  10.0  79233165           4832.51          429.0   246.0    INFO     PFEN El dataframe cargado es:  1116-1117
   No.  Contrato  Capital Insoluto  Dias Vencidos    DSM Mercado Producto
0  1.0  72448658          78849.69          273.0  190

In [134]:
import pandas as pd

# Lista para guardar los DataFrames etiquetados
lista_df = []

for nombre, df in dataframes.items():
    df_copy = df.copy()  # evitar modificar el original
    df_copy["Venta"] = nombre  # agregar la columna con el nombre del archivo
    lista_df.append(df_copy)

# Concatenar todos en uno solo
df_consolidado = pd.concat(lista_df, ignore_index=True)


In [135]:
df_consolidado = df_consolidado[["No.",	"Contrato",	"Capital Insoluto",	"Dias Vencidos", "DSM",	"Mercado", "Producto", "Venta"]]

In [136]:
len(df_consolidado)

17

In [137]:
lista_contratos = df_consolidado["Contrato"].dropna().astype(int).tolist()

In [138]:
texto = ', '.join([str(c) for c in lista_contratos])

In [139]:
print(texto)

75248092, 78406574, 79058845, 79072473, 79115128, 79145900, 79171631, 79183073, 79208882, 79233165, 72448658, 77925304, 78163518, 78280702, 78896360, 79190588, 79217065


In [140]:
import os
from google.cloud import bigquery ##, storage
import pandas as pd

from google.cloud import bigquery

# Reemplaza con tu ID de proyecto real
client = bigquery.Client(project="findep-riesgos")

c:\Users\apaloalto\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\auth\_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [141]:
#client = bigquery.Client()
query = f"""
select E.sucursal, A.contrato, E.cliente, concat(A.Nombre_cliente, " ", A.Apellido_paterno_cliente, " ", A.Apellido_materno_cliente) AS NOMBRE, "CJEX" AS  propietario_relacion_abogados,
E.dias_sin_movimiento + 1 as dias_sin_mov, E.dias_vencidos + 1 as dias_vencidos, E.segmento, E.capital_insoluto, E.id_contrato_migrado
from `BUO_FISA.evolucion_diaria_contratos` AS E
right join `Logisitica.fisa_altas_y_saldos_{mes_anio}` AS A
ON A.contrato = CAST(E.contrato AS NUMERIC) 
WHERE fecha_informacion = "{fecha_truncada}"
AND A.contrato in ({texto});"""

df = client.query(query).to_dataframe()
print(df)

c:\Users\apaloalto\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


    sucursal  contrato       cliente                                 NOMBRE  \
0         28  79115128  000724241595          ANDRES ENRIQUE DZUL HERNANDEZ   
1         28  79233165  000728415626          TERESA ESTHEFANIA ORTIZ REYES   
2        277  77925304  000724296047                 JOSE LUIS VALDEZ VILLA   
3        288  79171631  000728102906                GERARDO ERNESTO CHI PUC   
4         28  75248092  000722881922                     ELVIA CAUICH CANUL   
5        277  78280702  000550766728                JOSE LUIS MARTINEZ VEGA   
6        288  79145900  000727010063            WENDI LLESEÑI CHE HERNANDEZ   
7        275  72448658  000720338603           LAURA SILVIA PEREZ HERNANDEZ   
8        288  79058845  000727385656       JAQUELIN RADAI PEDRERO VELAZQUEZ   
9        277  78896360  000721939731                OMAR JOSE PINTOR PINEDA   
10       288  79183073  000726228590                    ABIEZER LARA ZAPATA   
11       288  79208882  000728298057  VANESSA DE LOS

In [142]:
df[df["contrato"] == 79026664]

,sucursal,contrato,cliente,NOMBRE,propietario_relacion_abogados,dias_sin_mov,dias_vencidos,segmento,capital_insoluto,id_contrato_migrado


In [143]:
#client = bigquery.Client()
query2 = f"""
select * FROM `Castigos_FISA.Concentrado_FISA_2025`;"""

concen = client.query(query2).to_dataframe()


c:\Users\apaloalto\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [144]:
concen.tail()

,No,SUCURSAL,No_de_Suc,Centro_de_costos,Division,Fecha_contrato_Fecha_de_Pago,No_De_contratos,Capital_insoluto_total,Prom_DV,Valor_de_venta_Precio_de_venta,...,DemandasAvance_procesal,Ilocalizables,Restituciones_otras,Subdireccion,Solicito_Factura,Vacia,FECHA_Y_HORA_DE_ENTREGA,Observaciones,EJECUTIVO,No_DE_CUENTA
2109,<NA>,None,None,None,None,NaT,None,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2110,<NA>,None,None,None,None,NaT,None,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2111,<NA>,None,None,None,None,NaT,None,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2112,<NA>,None,None,None,None,NaT,None,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2113,<NA>,None,None,None,None,NaT,None,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


In [145]:
concen["No"].describe()

count        1121.0
mean          561.0
std      323.749131
min             1.0
25%           281.0
50%           561.0
75%           841.0
max          1121.0
Name: No, dtype: Float64

### Cargaremos los valores de la base

In [146]:
teacher = df

In [147]:
teacher.head()

,sucursal,contrato,cliente,NOMBRE,propietario_relacion_abogados,dias_sin_mov,dias_vencidos,segmento,capital_insoluto,id_contrato_migrado
0,28,79115128,000724241595,ANDRES ENRIQUE DZUL HERNANDEZ,CJEX,<NA>,850,INFORMAL,6256.73,None
1,28,79233165,000728415626,TERESA ESTHEFANIA ORTIZ REYES,CJEX,<NA>,430,INFORMAL,4832.51,None
2,277,77925304,000724296047,JOSE LUIS VALDEZ VILLA,CJEX,<NA>,300,INFORMAL,23613.58,None
3,288,79171631,000728102906,GERARDO ERNESTO CHI PUC,CJEX,<NA>,606,INFORMAL,6098.92,None
4,28,75248092,000722881922,ELVIA CAUICH CANUL,CJEX,<NA>,500,INFORMAL,20736.49,None


In [148]:
teacher.head()

,sucursal,contrato,cliente,NOMBRE,propietario_relacion_abogados,dias_sin_mov,dias_vencidos,segmento,capital_insoluto,id_contrato_migrado
0,28,79115128,000724241595,ANDRES ENRIQUE DZUL HERNANDEZ,CJEX,<NA>,850,INFORMAL,6256.73,None
1,28,79233165,000728415626,TERESA ESTHEFANIA ORTIZ REYES,CJEX,<NA>,430,INFORMAL,4832.51,None
2,277,77925304,000724296047,JOSE LUIS VALDEZ VILLA,CJEX,<NA>,300,INFORMAL,23613.58,None
3,288,79171631,000728102906,GERARDO ERNESTO CHI PUC,CJEX,<NA>,606,INFORMAL,6098.92,None
4,28,75248092,000722881922,ELVIA CAUICH CANUL,CJEX,<NA>,500,INFORMAL,20736.49,None


In [149]:
teacher["dias_vencidos"].describe()

count          17.0
mean     561.823529
std       342.02453
min           274.0
25%           362.0
50%           430.0
75%           606.0
max          1625.0
Name: dias_vencidos, dtype: Float64

In [150]:
import numpy as np

# Definir las condiciones
conditions = [
    (teacher['dias_vencidos'] >= 180) & (teacher['dias_vencidos'] <= 360),
    (teacher['dias_vencidos'] >= 361) & (teacher['dias_vencidos'] <= 540),
    (teacher['dias_vencidos'] >= 541) & (teacher['dias_vencidos'] <= 720),
    (teacher['dias_vencidos'] >= 721) & (teacher['dias_vencidos'] <= 900),
    (teacher['dias_vencidos'] >= 901) & (teacher['dias_vencidos'] <= 1080),
    (teacher['dias_vencidos'] >= 1081) & (teacher['dias_vencidos'] <= 1260),
    (teacher['dias_vencidos'] >= 1261) & (teacher['dias_vencidos'] <= 1440),
    (teacher['dias_vencidos'] >= 1441)
]

# Etiquetas correspondientes
buckets = ['bk1', 'bk2', 'bk3', 'bk4', 'bk5', 'bk6', 'bk7', 'fenix']

# Aplicar la clasificación
teacher['Bucket'] = np.select(conditions, buckets, default='Sin Clasificar')


In [151]:
teacher["Bucket"].value_counts(dropna=False)

Bucket
bk2      9
bk3      3
bk1      2
bk4      1
bk5      1
fenix    1
Name: count, dtype: int64

In [152]:
teacher["contrato"].dtype

Int64Dtype()

In [153]:
##teacher["contrato"] = teacher["contrato"].astype(str).str.strip()

In [154]:
precios = pd.read_excel("./Consulta/Precios_FISA.xlsx")

In [155]:
precios.head(2)

,Bucket,Rango DV inferior,Rango DV superior,Precio comercial,Precio CON pagare,Precio SIN pagare o CON error
0,bk1,180,360,0.18,0.150,0.122
1,bk2,361,540,0.11,0.098,0.049


In [156]:
teacher = teacher.merge(precios[["Bucket","Precio CON pagare"]],how="left",on="Bucket")

In [157]:
teacher["Valor_Venta"] = teacher["capital_insoluto"] * teacher["Precio CON pagare"]

In [158]:
teacher.head()

,sucursal,contrato,cliente,NOMBRE,propietario_relacion_abogados,dias_sin_mov,dias_vencidos,segmento,capital_insoluto,id_contrato_migrado,Bucket,Precio CON pagare,Valor_Venta
0,28,79115128,000724241595,ANDRES ENRIQUE DZUL HERNANDEZ,CJEX,<NA>,850,INFORMAL,6256.73,None,bk4,0.056,350.37688
1,28,79233165,000728415626,TERESA ESTHEFANIA ORTIZ REYES,CJEX,<NA>,430,INFORMAL,4832.51,None,bk2,0.098,473.58598
2,277,77925304,000724296047,JOSE LUIS VALDEZ VILLA,CJEX,<NA>,300,INFORMAL,23613.58,None,bk1,0.150,3542.03700
3,288,79171631,000728102906,GERARDO ERNESTO CHI PUC,CJEX,<NA>,606,INFORMAL,6098.92,None,bk3,0.072,439.12224
4,28,75248092,000722881922,ELVIA CAUICH CANUL,CJEX,<NA>,500,INFORMAL,20736.49,None,bk2,0.098,2032.17602


In [159]:
teacher.dtypes

sucursal                           Int64
contrato                           Int64
cliente                           object
NOMBRE                            object
propietario_relacion_abogados     object
dias_sin_mov                       Int64
dias_vencidos                      Int64
segmento                          object
capital_insoluto                 float64
id_contrato_migrado               object
Bucket                            object
Precio CON pagare                float64
Valor_Venta                      float64
dtype: object

In [160]:
df_consolidado.head()

,No.,Contrato,Capital Insoluto,Dias Vencidos,DSM,Mercado,Producto,Venta
0,1.0,75248092,20736.49,499.0,367.0,INFO,POPN,1116-1117
1,2.0,78406574,32290.93,1624.0,1541.0,FORM,REVN,1116-1117
2,3.0,79058845,9487.58,1046.0,400.0,INFO,PFEN,1116-1117
3,4.0,79072473,26942.41,361.0,153.0,FORM,REVN,1116-1117
4,5.0,79115128,6256.73,849.0,644.0,INFO,PFEN,1116-1117


In [161]:
df_consolidado[df_consolidado["Contrato"] == 79147694]

,No.,Contrato,Capital Insoluto,Dias Vencidos,DSM,Mercado,Producto,Venta


In [162]:
df_consolidado["Contrato"] = pd.to_numeric(df_consolidado["Contrato"], errors="coerce").astype("Int64")

#df_consolidado["Contrato"] = pd.to_numeric(df_consolidado["Contrato"],errors="coerce")
#df_consolidado["Contrato"] = pd.to_numeric(df_consolidado["Contrato"], errors="coerce")


In [163]:
#df_consolidado["Contrato"] = df_consolidado["Contrato"].astype(str)

In [164]:
df_consolidado.Contrato.dtypes

Int64Dtype()

In [165]:
df_consolidado["Contrato"].value_counts()

Contrato
75248092    1
78406574    1
79058845    1
79072473    1
79115128    1
79145900    1
79171631    1
79183073    1
79208882    1
79233165    1
72448658    1
77925304    1
78163518    1
78280702    1
78896360    1
79190588    1
79217065    1
Name: count, dtype: Int64

In [166]:
#df_consolidado["Contrato"] = df_consolidado["Contrato"].astype(str).str.strip()

In [167]:
df_consolidado["Contrato"].head()

0    75248092
1    78406574
2    79058845
3    79072473
4    79115128
Name: Contrato, dtype: Int64

In [168]:
teacher.head()

,sucursal,contrato,cliente,NOMBRE,propietario_relacion_abogados,dias_sin_mov,dias_vencidos,segmento,capital_insoluto,id_contrato_migrado,Bucket,Precio CON pagare,Valor_Venta
0,28,79115128,000724241595,ANDRES ENRIQUE DZUL HERNANDEZ,CJEX,<NA>,850,INFORMAL,6256.73,None,bk4,0.056,350.37688
1,28,79233165,000728415626,TERESA ESTHEFANIA ORTIZ REYES,CJEX,<NA>,430,INFORMAL,4832.51,None,bk2,0.098,473.58598
2,277,77925304,000724296047,JOSE LUIS VALDEZ VILLA,CJEX,<NA>,300,INFORMAL,23613.58,None,bk1,0.150,3542.03700
3,288,79171631,000728102906,GERARDO ERNESTO CHI PUC,CJEX,<NA>,606,INFORMAL,6098.92,None,bk3,0.072,439.12224
4,28,75248092,000722881922,ELVIA CAUICH CANUL,CJEX,<NA>,500,INFORMAL,20736.49,None,bk2,0.098,2032.17602


In [169]:
#teacher.merge(df_consolidado)
teacher = teacher.merge(df_consolidado[["Contrato","Capital Insoluto","Venta","Dias Vencidos"]],how="left",left_on="contrato",right_on="Contrato")

In [170]:
len(teacher[teacher["Contrato"].isna()])

0

In [171]:
teacher["Val_contrato"] = teacher["contrato"] == teacher["Contrato"]

In [172]:
teacher["Val_contrato"].value_counts(dropna=False)

Val_contrato
True    17
Name: count, dtype: Int64

In [173]:
teacher["Val_cap"] = teacher["capital_insoluto"] == teacher["Capital Insoluto"]

In [174]:
teacher["Val_cap"].value_counts(dropna=False)

Val_cap
True    17
Name: count, dtype: int64

In [175]:
teacher[teacher["Val_cap"] == False][["contrato","capital_insoluto", "Capital Insoluto"]]

,contrato,capital_insoluto,Capital Insoluto


In [176]:
teacher["Venta"].unique()

array(['1116-1117', '1118-1119-1120-1121'], dtype=object)

In [177]:
concentrado2 = concen

In [178]:
concentrado = concen

In [179]:
concentrado.columns

Index(['No', 'SUCURSAL', 'No_de_Suc', 'Centro_de_costos', 'Division',
       'Fecha_contrato_Fecha_de_Pago', 'No_De_contratos',
       'Capital_insoluto_total', 'Prom_DV', 'Valor_de_venta_Precio_de_venta',
       'Precio', 'ValuacionGE', 'Diferencia', 'Fecha_baja_sistema',
       'Mes_aplicacion', 'MES_DEPOSITO', 'COMPRADOR', 'No_pagos_pactados',
       'Pagos_realizados', 'Depositos', 'Saldo', 'DemandasAvance_procesal',
       'Ilocalizables', 'Restituciones_otras', 'Subdireccion',
       'Solicito_Factura', 'Vacia', 'FECHA_Y_HORA_DE_ENTREGA', 'Observaciones',
       'EJECUTIVO', 'No_DE_CUENTA'],
      dtype='object')

In [180]:
concentrado = concentrado[["No","SUCURSAL","No_de_Suc", "No_De_contratos","Capital_insoluto_total","Prom_DV",
                           "Valor_de_venta_Precio_de_venta","Fecha_contrato_Fecha_de_Pago","Precio","COMPRADOR"]]

In [181]:
concentrado['No'] = pd.to_numeric(concentrado['No'], errors='coerce')

C:\Users\apaloalto\AppData\Local\Temp\ipykernel_24624\2597250597.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concentrado['No'] = pd.to_numeric(concentrado['No'], errors='coerce')


In [182]:
concentrado['No'] = concentrado['No'].fillna(0).astype(int)
concentrado['No'] = concentrado['No'].astype(int)

C:\Users\apaloalto\AppData\Local\Temp\ipykernel_24624\2815349332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concentrado['No'] = concentrado['No'].fillna(0).astype(int)
C:\Users\apaloalto\AppData\Local\Temp\ipykernel_24624\2815349332.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concentrado['No'] = concentrado['No'].astype(int)


In [183]:
concentrado.columns

Index(['No', 'SUCURSAL', 'No_de_Suc', 'No_De_contratos',
       'Capital_insoluto_total', 'Prom_DV', 'Valor_de_venta_Precio_de_venta',
       'Fecha_contrato_Fecha_de_Pago', 'Precio', 'COMPRADOR'],
      dtype='object')

In [184]:
concentrado = concentrado.rename(columns={"No_De_contratos":"No. De contratos_concentrado","Capital_insoluto_total":"Capital concentrado",
                           "Prom_DV":"Prom. DV. Concentrado"})

In [185]:
concentrado.head()

,No,SUCURSAL,No_de_Suc,No. De contratos_concentrado,Capital concentrado,Prom. DV. Concentrado,Valor_de_venta_Precio_de_venta,Fecha_contrato_Fecha_de_Pago,Precio,COMPRADOR
0,1,TEPIC PLAZA LA CANTERA,294,3,77014.07,240.000000,14000.000000,2025-01-10,0.180000000,RAUL AVILA RODRIGUEZ
1,2,IRAPUATO 1,5,13,214381.99,455.461539,14826.942460,2025-01-13,0.070000000,CLAUDIA LORENA CHACON LUNA
2,3,IRAPUATO LAS ARBOLEDAS,295,4,74796.97,496.500000,5173.057542,2025-01-13,0.070000000,CLAUDIA LORENA CHACON LUNA
3,4,CELAYA 1,2,1,48529.86,314.000000,5434.330192,2025-01-13,0.110000000,CUAHUTEMOC ONTIVEROS MORALES
4,5,CELAYA / CORTAZAR,264,3,33634.36,380.333333,3766.345463,2025-01-13,0.110000000,CUAHUTEMOC ONTIVEROS MORALES


In [186]:
concentrado.head(2)

,No,SUCURSAL,No_de_Suc,No. De contratos_concentrado,Capital concentrado,Prom. DV. Concentrado,Valor_de_venta_Precio_de_venta,Fecha_contrato_Fecha_de_Pago,Precio,COMPRADOR
0,1,TEPIC PLAZA LA CANTERA,294,3,77014.07,240.000000,14000.00000,2025-01-10,0.180000000,RAUL AVILA RODRIGUEZ
1,2,IRAPUATO 1,5,13,214381.99,455.461539,14826.94246,2025-01-13,0.070000000,CLAUDIA LORENA CHACON LUNA


In [187]:
teacher[teacher["Venta"] == "1083-1084"]["capital_insoluto"].sum()

np.float64(0.0)

In [188]:
# Aseguramos consistencia de tipos
teacher['Venta'] = teacher['Venta'].astype(str)
teacher['sucursal'] = teacher['sucursal'].astype(str)
concentrado['No'] = concentrado['No'].astype(str)
concentrado['No_de_Suc'] = concentrado['No_de_Suc'].astype(str)

# Crear conjunto de ventas válidas con sucursal
ventas_sucursales = set(zip(concentrado['No'], concentrado['No_de_Suc']))

# Función para desambiguar la venta
def buscar_venta_valida(ventas_str, sucursal):
    posibles = ventas_str.split('-')
    for venta in posibles:
        par = (venta.strip(), sucursal.strip())
        if par in ventas_sucursales:
            return venta.strip()
    return None  # No se encontró coincidencia

# Aplicar función fila por fila
teacher['Venta_real'] = teacher.apply(
    lambda row: buscar_venta_valida(row['Venta'], row['sucursal']), axis=1
)


In [189]:
teacher.head()

,sucursal,contrato,cliente,NOMBRE,propietario_relacion_abogados,dias_sin_mov,dias_vencidos,segmento,capital_insoluto,id_contrato_migrado,Bucket,Precio CON pagare,Valor_Venta,Contrato,Capital Insoluto,Venta,Dias Vencidos,Val_contrato,Val_cap,Venta_real
0,28,79115128,000724241595,ANDRES ENRIQUE DZUL HERNANDEZ,CJEX,<NA>,850,INFORMAL,6256.73,None,bk4,0.056,350.37688,79115128,6256.73,1116-1117,849.0,True,True,1116
1,28,79233165,000728415626,TERESA ESTHEFANIA ORTIZ REYES,CJEX,<NA>,430,INFORMAL,4832.51,None,bk2,0.098,473.58598,79233165,4832.51,1116-1117,429.0,True,True,1116
2,277,77925304,000724296047,JOSE LUIS VALDEZ VILLA,CJEX,<NA>,300,INFORMAL,23613.58,None,bk1,0.150,3542.03700,77925304,23613.58,1118-1119-1120-1121,299.0,True,True,1121
3,288,79171631,000728102906,GERARDO ERNESTO CHI PUC,CJEX,<NA>,606,INFORMAL,6098.92,None,bk3,0.072,439.12224,79171631,6098.92,1116-1117,605.0,True,True,1117
4,28,75248092,000722881922,ELVIA CAUICH CANUL,CJEX,<NA>,500,INFORMAL,20736.49,None,bk2,0.098,2032.17602,75248092,20736.49,1116-1117,499.0,True,True,1116


In [190]:
concentrado.dtypes

No                                 object
SUCURSAL                           object
No_de_Suc                          object
No. De contratos_concentrado       object
Capital concentrado               float64
Prom. DV. Concentrado             float64
Valor_de_venta_Precio_de_venta    float64
Fecha_contrato_Fecha_de_Pago       dbdate
Precio                             object
COMPRADOR                          object
dtype: object

In [191]:
teacher = teacher.merge(concentrado[["No","Fecha_contrato_Fecha_de_Pago","SUCURSAL","COMPRADOR"]],
                        how="left",left_on="Venta_real",right_on="No")

In [192]:
teacher['sucursal'] = teacher['sucursal'].astype(int)

In [193]:
teacher[teacher["contrato"] == "79150134"]

,sucursal,contrato,cliente,NOMBRE,propietario_relacion_abogados,dias_sin_mov,dias_vencidos,segmento,capital_insoluto,id_contrato_migrado,...,Capital Insoluto,Venta,Dias Vencidos,Val_contrato,Val_cap,Venta_real,No,Fecha_contrato_Fecha_de_Pago,SUCURSAL,COMPRADOR


In [194]:
pivot = teacher.pivot_table(
    index='Venta_real',
    values=['capital_insoluto', 'dias_vencidos','sucursal'],
    aggfunc={
        'capital_insoluto': 'sum',
        'dias_vencidos': 'mean',
        'sucursal': 'mean'
    }
)

pivot['Cuentas'] = teacher.groupby('Venta_real').size()


In [195]:
pivot["sucursal"] = pivot["sucursal"].astype(int)

In [196]:
pivot["dias_vencidos"] = pivot["dias_vencidos"].astype(int)

In [197]:
pivot = pivot.reset_index()

In [198]:
pivot

,Venta_real,capital_insoluto,dias_vencidos,sucursal,Cuentas
0,1116,91059.07,753,28,5
1,1117,48863.43,680,288,5
2,1118,54065.37,362,255,1
3,1119,44632.48,362,257,1
4,1120,78849.69,274,275,1
5,1121,188848.77,346,277,4


In [199]:
pivot[pivot["Venta_real"] == 1084]

,Venta_real,capital_insoluto,dias_vencidos,sucursal,Cuentas


In [200]:
teacher[teacher["Venta_real"] == 859]

,sucursal,contrato,cliente,NOMBRE,propietario_relacion_abogados,dias_sin_mov,dias_vencidos,segmento,capital_insoluto,id_contrato_migrado,...,Capital Insoluto,Venta,Dias Vencidos,Val_contrato,Val_cap,Venta_real,No,Fecha_contrato_Fecha_de_Pago,SUCURSAL,COMPRADOR


In [201]:
concentrado.head(1)

,No,SUCURSAL,No_de_Suc,No. De contratos_concentrado,Capital concentrado,Prom. DV. Concentrado,Valor_de_venta_Precio_de_venta,Fecha_contrato_Fecha_de_Pago,Precio,COMPRADOR
0,1,TEPIC PLAZA LA CANTERA,294,3,77014.07,240.0,14000.0,2025-01-10,0.180000000,RAUL AVILA RODRIGUEZ


In [202]:
concentrado["No. De contratos_concentrado"] = concentrado["No. De contratos_concentrado"].fillna(0).astype(int)
concentrado["No. De contratos_concentrado"] = concentrado["No. De contratos_concentrado"].astype(int)

In [203]:
concentrado["Prom. DV. Concentrado"] = concentrado["Prom. DV. Concentrado"].fillna(0).astype(int)
concentrado["Prom. DV. Concentrado"] = concentrado["Prom. DV. Concentrado"].astype(int)

In [204]:
concentrado.head(1)

,No,SUCURSAL,No_de_Suc,No. De contratos_concentrado,Capital concentrado,Prom. DV. Concentrado,Valor_de_venta_Precio_de_venta,Fecha_contrato_Fecha_de_Pago,Precio,COMPRADOR
0,1,TEPIC PLAZA LA CANTERA,294,3,77014.07,240,14000.0,2025-01-10,0.180000000,RAUL AVILA RODRIGUEZ


In [205]:
validaciones = pivot.merge(concentrado[["No","No_de_Suc","No. De contratos_concentrado", "Capital concentrado","Prom. DV. Concentrado"]],how="left",left_on="Venta_real",right_on="No")

In [206]:
validaciones.head()

,Venta_real,capital_insoluto,dias_vencidos,sucursal,Cuentas,No,No_de_Suc,No. De contratos_concentrado,Capital concentrado,Prom. DV. Concentrado
0,1116,91059.07,753,28,5,1116,28,5,91059.07,782
1,1117,48863.43,680,288,5,1117,288,5,48863.43,709
2,1118,54065.37,362,255,1,1118,255,1,54065.37,391
3,1119,44632.48,362,257,1,1119,257,1,44632.48,391
4,1120,78849.69,274,275,1,1120,275,1,78849.69,303


In [207]:
validaciones["val_cap"] = validaciones["capital_insoluto"] - validaciones["Capital concentrado"]

In [208]:
conteo = validaciones['val_cap'].value_counts(dropna=False)
conteo.index = conteo.index.round()
print(conteo)

val_cap
0.0    5
0.0    1
Name: count, dtype: int64


In [209]:
validaciones[validaciones["val_cap"] > 1]

,Venta_real,capital_insoluto,dias_vencidos,sucursal,Cuentas,No,No_de_Suc,No. De contratos_concentrado,Capital concentrado,Prom. DV. Concentrado,val_cap


In [210]:
validaciones[validaciones["val_cap"] < -1]

,Venta_real,capital_insoluto,dias_vencidos,sucursal,Cuentas,No,No_de_Suc,No. De contratos_concentrado,Capital concentrado,Prom. DV. Concentrado,val_cap


In [211]:
teacher[(teacher["Venta_real"] == "795") & (teacher["contrato"] == "79150134")]

,sucursal,contrato,cliente,NOMBRE,propietario_relacion_abogados,dias_sin_mov,dias_vencidos,segmento,capital_insoluto,id_contrato_migrado,...,Capital Insoluto,Venta,Dias Vencidos,Val_contrato,Val_cap,Venta_real,No,Fecha_contrato_Fecha_de_Pago,SUCURSAL,COMPRADOR


In [212]:
import numpy as np
condicion_critica = np.where(len(validaciones[(validaciones["val_cap"] > 1) |  (validaciones["val_cap"] > 1) ]) >= 1, True,False)

In [213]:
condicion_critica

array(False)

In [214]:
if condicion_critica:
    from IPython.display import display, HTML
    display(HTML("<div style='color:red; font-weight:bold;'>⚠️ Error: condición inválida</div>"))
    raise RuntimeError("Detenido por condición inválida")


In [215]:
concentrado.dtypes

No                                 object
SUCURSAL                           object
No_de_Suc                          object
No. De contratos_concentrado        int64
Capital concentrado               float64
Prom. DV. Concentrado               int64
Valor_de_venta_Precio_de_venta    float64
Fecha_contrato_Fecha_de_Pago       dbdate
Precio                             object
COMPRADOR                          object
dtype: object

In [216]:
from openpyxl import load_workbook
from datetime import datetime
import locale
import os
from datetime import date

fecha_actual = date.today()
subcarpeta = fecha_actual.strftime('%d%m')  # Día + Mes → '2207'

ruta_guardado = f"./Resultado_programa/Diciembre/{subcarpeta}"

os.makedirs(ruta_guardado, exist_ok=True)

# Establecer el idioma a español (ajusta según tu sistema operativo)
try:
    locale.setlocale(locale.LC_TIME, 'Spanish_Spain')
except:
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

ventas_unicas = teacher['Venta_real'].dropna().unique()

for venta in ventas_unicas:
    venta_str = str(venta)
    
    # ➤ Obtener valores base
    sucursal_valor = concentrado[concentrado["No"] == venta_str]["SUCURSAL"].astype(str).tolist()
    fecha_valor = concentrado[concentrado["No"] == venta_str]["Fecha_contrato_Fecha_de_Pago"].astype(str).tolist()
    
    if not sucursal_valor or not fecha_valor:
        continue  # saltar si falta información

    # ➤ Formatear fecha
    fecha_formateada = datetime.strptime(fecha_valor[0], '%Y-%m-%d').strftime('%d %B %Y')
    
    # ➤ Encabezado
    Encabezado = f"Venta de cartera Sucursal {sucursal_valor[0]} {fecha_formateada}"
    
    # ➤ Filtrar cuentas
    cuentas = teacher[teacher["Venta_real"] == venta_str][[
        "sucursal", "contrato", "capital_insoluto", "Dias Vencidos", "Precio CON pagare", "Valor_Venta"
    ]].copy()

    # ➤ Calcular columnas y métricas
    precio_valor = concentrado[concentrado["No"] == venta_str]["Precio"].values[0]

    precio_valor = float(precio_valor)
    
    cuentas["Venta Real"] = cuentas["capital_insoluto"] * precio_valor

    cap_suma = cuentas["capital_insoluto"].sum()
    prom_dv = cuentas["Dias Vencidos"].mean()
    valor_ge_sum = cuentas["Valor_Venta"].sum()
    venta_real_sum = cuentas["Venta Real"].sum()
    diferencia = venta_real_sum - valor_ge_sum

    # ➤ Exportar a Excel base
    wb = load_workbook("./Muestra.xlsx")
    ws = wb.active

    ws['H1'].value = f"V-{venta_str}"
    ws['A3'].value = Encabezado

    start_row = 6
    start_col = 2

    for i, row in enumerate(cuentas.values):
        for j, value in enumerate(row):
            ws.cell(row=start_row + i, column=start_col + j, value=value)

    fila_final = start_row + len(cuentas)

    if fila_final < 568:
        ws.delete_rows(fila_final, 569 - fila_final)

    ws.cell(fila_final, 4, cap_suma)
    ws.cell(fila_final, 5, prom_dv)
    ws.cell(fila_final, 7, valor_ge_sum)
    ws.cell(fila_final, 8, venta_real_sum)
    ws.cell(fila_final + 1, 8, diferencia)

    # ➤ Guardar archivo
    wb.save(f"./{ruta_guardado}/VTA - {venta_str}.xlsx")
    print("Se guardo archivo de venta",venta_str)

    #nombre_archivo = f"VTA - {venta_str}.xlsx"
    #wb.save(nombre_archivo)


Se guardo archivo de venta 1116
Se guardo archivo de venta 1121
Se guardo archivo de venta 1117
Se guardo archivo de venta 1120
Se guardo archivo de venta 1118
Se guardo archivo de venta 1119


In [217]:
teacher.head()

,sucursal,contrato,cliente,NOMBRE,propietario_relacion_abogados,dias_sin_mov,dias_vencidos,segmento,capital_insoluto,id_contrato_migrado,...,Capital Insoluto,Venta,Dias Vencidos,Val_contrato,Val_cap,Venta_real,No,Fecha_contrato_Fecha_de_Pago,SUCURSAL,COMPRADOR
0,28,79115128,000724241595,ANDRES ENRIQUE DZUL HERNANDEZ,CJEX,<NA>,850,INFORMAL,6256.73,None,...,6256.73,1116-1117,849.0,True,True,1116,1116,2025-12-10,CAMPECHE,MANUEL ALEJANDRO CHUC HERNANDEZ
1,28,79233165,000728415626,TERESA ESTHEFANIA ORTIZ REYES,CJEX,<NA>,430,INFORMAL,4832.51,None,...,4832.51,1116-1117,429.0,True,True,1116,1116,2025-12-10,CAMPECHE,MANUEL ALEJANDRO CHUC HERNANDEZ
2,277,77925304,000724296047,JOSE LUIS VALDEZ VILLA,CJEX,<NA>,300,INFORMAL,23613.58,None,...,23613.58,1118-1119-1120-1121,299.0,True,True,1121,1121,2025-12-30,ESTADIO,ROGER AUGUSTO RODRIGUEZ MARTINEZ
3,288,79171631,000728102906,GERARDO ERNESTO CHI PUC,CJEX,<NA>,606,INFORMAL,6098.92,None,...,6098.92,1116-1117,605.0,True,True,1117,1117,2025-12-10,CAMPECHE PLAZA CRYSTALS,MANUEL ALEJANDRO CHUC HERNANDEZ
4,28,75248092,000722881922,ELVIA CAUICH CANUL,CJEX,<NA>,500,INFORMAL,20736.49,None,...,20736.49,1116-1117,499.0,True,True,1116,1116,2025-12-10,CAMPECHE,MANUEL ALEJANDRO CHUC HERNANDEZ


In [218]:
from datetime import date

hoy = date.today()
#print(hoy)  # Ejemplo: 2025-07-23


In [219]:
ventas_aplicar = teacher[["sucursal",'contrato', 'capital_insoluto','Fecha_contrato_Fecha_de_Pago',"Venta_real"]]

In [220]:
ventas_aplicar

,sucursal,contrato,capital_insoluto,Fecha_contrato_Fecha_de_Pago,Venta_real
0,28,79115128,6256.73,2025-12-10,1116
1,28,79233165,4832.51,2025-12-10,1116
2,277,77925304,23613.58,2025-12-30,1121
3,288,79171631,6098.92,2025-12-10,1117
4,28,75248092,20736.49,2025-12-10,1116
5,277,78280702,31640.00,2025-12-30,1121
6,288,79145900,5808.31,2025-12-10,1117
7,275,72448658,78849.69,2025-12-30,1120
8,288,79058845,9487.58,2025-12-10,1117
9,277,78896360,51133.86,2025-12-30,1121


In [221]:
import os

fecha_str = hoy.strftime('%y%m%d')

# Crear carpeta si no existe
os.makedirs("Aplicar", exist_ok=True)

# Guardar el archivo con la fecha en el nombre
ventas_aplicar.to_excel(f"./Aplicar/Ventas_a_aplicar_{fecha_str}.xlsx", index=False)


In [222]:
ventas_aplicar.columns

Index(['sucursal', 'contrato', 'capital_insoluto',
       'Fecha_contrato_Fecha_de_Pago', 'Venta_real'],
      dtype='object')

In [223]:
#client = bigquery.Client()
query3 = f"""
select * FROM `Castigos_FISA.LiquidacionesFISA_BQ`;"""

liq = client.query(query3).to_dataframe()

c:\Users\apaloalto\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [224]:
liq.head()

,Credito,Mesdepago
0,78816340,None
1,78874107,None
2,78882828,None
3,78954182,None
4,79000631,None


In [225]:
liq = liq.rename(columns={"Credito":"Crédito"})

In [226]:
liq["Crédito"] = liq["Crédito"].astype(str).str.strip()

In [227]:
liq["Crédito"] = pd.to_numeric(liq["Crédito"], errors="coerce").astype("Int64")


In [228]:
ventas_aplicar = ventas_aplicar.merge(liq["Crédito"],how="left",left_on="contrato",right_on="Crédito")

In [229]:
ventas_aplicar["Crédito"].value_counts(dropna=False)

Crédito
<NA>    17
Name: count, dtype: Int64

In [230]:
print("Tenemos el siguiente numero de cuentas liquidadas que no se pueden vender: ", len(ventas_aplicar[ventas_aplicar["Crédito"].notna()]))

Tenemos el siguiente numero de cuentas liquidadas que no se pueden vender:  0


### Generacion Archivo Luis

In [231]:
teacher.head()

,sucursal,contrato,cliente,NOMBRE,propietario_relacion_abogados,dias_sin_mov,dias_vencidos,segmento,capital_insoluto,id_contrato_migrado,...,Capital Insoluto,Venta,Dias Vencidos,Val_contrato,Val_cap,Venta_real,No,Fecha_contrato_Fecha_de_Pago,SUCURSAL,COMPRADOR
0,28,79115128,000724241595,ANDRES ENRIQUE DZUL HERNANDEZ,CJEX,<NA>,850,INFORMAL,6256.73,None,...,6256.73,1116-1117,849.0,True,True,1116,1116,2025-12-10,CAMPECHE,MANUEL ALEJANDRO CHUC HERNANDEZ
1,28,79233165,000728415626,TERESA ESTHEFANIA ORTIZ REYES,CJEX,<NA>,430,INFORMAL,4832.51,None,...,4832.51,1116-1117,429.0,True,True,1116,1116,2025-12-10,CAMPECHE,MANUEL ALEJANDRO CHUC HERNANDEZ
2,277,77925304,000724296047,JOSE LUIS VALDEZ VILLA,CJEX,<NA>,300,INFORMAL,23613.58,None,...,23613.58,1118-1119-1120-1121,299.0,True,True,1121,1121,2025-12-30,ESTADIO,ROGER AUGUSTO RODRIGUEZ MARTINEZ
3,288,79171631,000728102906,GERARDO ERNESTO CHI PUC,CJEX,<NA>,606,INFORMAL,6098.92,None,...,6098.92,1116-1117,605.0,True,True,1117,1117,2025-12-10,CAMPECHE PLAZA CRYSTALS,MANUEL ALEJANDRO CHUC HERNANDEZ
4,28,75248092,000722881922,ELVIA CAUICH CANUL,CJEX,<NA>,500,INFORMAL,20736.49,None,...,20736.49,1116-1117,499.0,True,True,1116,1116,2025-12-10,CAMPECHE,MANUEL ALEJANDRO CHUC HERNANDEZ


In [232]:
teacher.columns

Index(['sucursal', 'contrato', 'cliente', 'NOMBRE',
       'propietario_relacion_abogados', 'dias_sin_mov', 'dias_vencidos',
       'segmento', 'capital_insoluto', 'id_contrato_migrado', 'Bucket',
       'Precio CON pagare', 'Valor_Venta', 'Contrato', 'Capital Insoluto',
       'Venta', 'Dias Vencidos', 'Val_contrato', 'Val_cap', 'Venta_real', 'No',
       'Fecha_contrato_Fecha_de_Pago', 'SUCURSAL', 'COMPRADOR'],
      dtype='object')

In [233]:
teacher.columns

Index(['sucursal', 'contrato', 'cliente', 'NOMBRE',
       'propietario_relacion_abogados', 'dias_sin_mov', 'dias_vencidos',
       'segmento', 'capital_insoluto', 'id_contrato_migrado', 'Bucket',
       'Precio CON pagare', 'Valor_Venta', 'Contrato', 'Capital Insoluto',
       'Venta', 'Dias Vencidos', 'Val_contrato', 'Val_cap', 'Venta_real', 'No',
       'Fecha_contrato_Fecha_de_Pago', 'SUCURSAL', 'COMPRADOR'],
      dtype='object')

In [234]:
Detalle = teacher[["Venta_real","SUCURSAL","Fecha_contrato_Fecha_de_Pago","COMPRADOR","Contrato","Capital Insoluto"]]

In [235]:
len(Detalle)

17

In [236]:
concentrado2.head()

,No,SUCURSAL,No_de_Suc,Centro_de_costos,Division,Fecha_contrato_Fecha_de_Pago,No_De_contratos,Capital_insoluto_total,Prom_DV,Valor_de_venta_Precio_de_venta,...,DemandasAvance_procesal,Ilocalizables,Restituciones_otras,Subdireccion,Solicito_Factura,Vacia,FECHA_Y_HORA_DE_ENTREGA,Observaciones,EJECUTIVO,No_DE_CUENTA
0,1,TEPIC PLAZA LA CANTERA,294,#N/A,#N/A,2025-01-10,3,77014.07,240.000000,14000.000000,...,None,None,None,Occidente,NO,None,None,None,NORMA,65502145958
1,2,IRAPUATO 1,5,205,Irapuato,2025-01-13,13,214381.99,455.461539,14826.942460,...,"6,000.00",None,"1,500.00",Centro,NO,None,None,None,JOSE,65502145958
2,3,IRAPUATO LAS ARBOLEDAS,295,#N/A,#N/A,2025-01-13,4,74796.97,496.500000,5173.057542,...,"6,000.00",None,"1,500.00",Centro,NO,None,None,None,JOSE,65502145958
3,4,CELAYA 1,2,202,Celaya,2025-01-13,1,48529.86,314.000000,5434.330192,...,None,None,None,Centro,NO,None,None,None,NANCY,65502145958
4,5,CELAYA / CORTAZAR,264,479,Celaya,2025-01-13,3,33634.36,380.333333,3766.345463,...,None,None,None,CENTRO,NO,None,None,None,NANCY,65502145958


In [237]:
import pandas as pd

teacher["Venta_real"] = pd.to_numeric(teacher["Venta_real"], errors="coerce")

venta_min = teacher["Venta_real"].min()

In [238]:
venta_max = teacher.Venta_real.max()

In [239]:
print("Las ventas de este dia son de la", venta_min,  "a la venta", venta_max)

Las ventas de este dia son de la 1116 a la venta 1121


In [240]:
type(venta_min)
type(venta_max)

numpy.int64

In [241]:
venta_min = int(venta_min)
venta_max = int(venta_max)

#concentrado['No.'] = pd.to_numeric(concentrado['No.'], errors='coerce')

In [242]:
concentrado2["No"] = pd.to_numeric(concentrado["No"],errors="coerce")

In [243]:
RESUMEN = concentrado2[(concentrado2["No"] <= venta_max) & (concentrado2["No"] >= venta_min)]

In [244]:
RESUMEN.head()

,No,SUCURSAL,No_de_Suc,Centro_de_costos,Division,Fecha_contrato_Fecha_de_Pago,No_De_contratos,Capital_insoluto_total,Prom_DV,Valor_de_venta_Precio_de_venta,...,DemandasAvance_procesal,Ilocalizables,Restituciones_otras,Subdireccion,Solicito_Factura,Vacia,FECHA_Y_HORA_DE_ENTREGA,Observaciones,EJECUTIVO,No_DE_CUENTA
1115,1116,CAMPECHE,28,627,Campeche,2025-12-10,5,91059.07,782.4,6507.821830,...,None,None,600.00,Peninsula,NO,None,None,None,JOSE,65502145958
1116,1117,CAMPECHE PLAZA CRYSTALS,288,725,Campeche,2025-12-10,5,48863.43,709.0,3492.178170,...,None,None,600.00,Peninsula,NO,None,None,None,JOSE,65502145958
1117,1118,BELENES,255,462,Guadalajara 1,2025-12-30,1,54065.37,391.0,7082.870895,...,None,None,None,Occidente,NO,None,None,None,JOSE,65502145958
1118,1119,PLAZA GIGANTES,257,465,Guadalajara 2,2025-12-30,1,44632.48,391.0,5847.108668,...,None,None,None,Occidente,NO,None,None,None,JOSE,65502145958
1119,1120,FEDERALISMO,275,509,Guadalajara 1,2025-12-30,1,78849.69,303.0,10329.757740,...,None,None,None,Occidente,NO,None,None,None,JOSE,65502145958


In [245]:
RESUMEN.columns

Index(['No', 'SUCURSAL', 'No_de_Suc', 'Centro_de_costos', 'Division',
       'Fecha_contrato_Fecha_de_Pago', 'No_De_contratos',
       'Capital_insoluto_total', 'Prom_DV', 'Valor_de_venta_Precio_de_venta',
       'Precio', 'ValuacionGE', 'Diferencia', 'Fecha_baja_sistema',
       'Mes_aplicacion', 'MES_DEPOSITO', 'COMPRADOR', 'No_pagos_pactados',
       'Pagos_realizados', 'Depositos', 'Saldo', 'DemandasAvance_procesal',
       'Ilocalizables', 'Restituciones_otras', 'Subdireccion',
       'Solicito_Factura', 'Vacia', 'FECHA_Y_HORA_DE_ENTREGA', 'Observaciones',
       'EJECUTIVO', 'No_DE_CUENTA'],
      dtype='object')

In [246]:
RESUMEN_EXP = RESUMEN[["No","SUCURSAL","Fecha_contrato_Fecha_de_Pago","COMPRADOR","No_De_contratos",
"Valor_de_venta_Precio_de_venta","Capital_insoluto_total"]]

In [247]:
len(RESUMEN_EXP)

6

In [248]:
RESUMEN_EXP.dtypes

No                                  int64
SUCURSAL                           object
Fecha_contrato_Fecha_de_Pago       dbdate
COMPRADOR                          object
No_De_contratos                    object
Valor_de_venta_Precio_de_venta    float64
Capital_insoluto_total            float64
dtype: object

In [249]:
RESUMEN_EXP["No_De_contratos"] = pd.to_numeric(RESUMEN_EXP["No_De_contratos"], errors="coerce").astype("Int64")


C:\Users\apaloalto\AppData\Local\Temp\ipykernel_24624\115331325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RESUMEN_EXP["No_De_contratos"] = pd.to_numeric(RESUMEN_EXP["No_De_contratos"], errors="coerce").astype("Int64")


In [250]:
RESUMEN_EXP["No_De_contratos"].sum()

np.int64(17)

In [251]:
import pandas as pd

ruta_excel = f"./Luis/Muestra_{fecha_str}.xlsx"

# Guardar ambos DataFrames en distintas hojas
with pd.ExcelWriter(ruta_excel, engine="openpyxl") as writer:
    RESUMEN.to_excel(writer, sheet_name="RESUMEN", index=False)  # Primera hoja
    Detalle.to_excel(writer, sheet_name="Detalles", index=False)  # Segunda hoja
#RESUMEN_EXP["No_De_contratos"].sum()

In [252]:
from openpyxl import load_workbook

ruta_guardado = "./Luis"
wb = load_workbook("./Muestra_Luis.xlsx")
ws1 = wb["RESUMEN"]
#ws = wb.active

start_row = 2  # A2 → fila 2
start_col = 1  #A -> columna 1
    
for i, row in enumerate(RESUMEN_EXP.values):
    for j, value in enumerate(row):
        ws1.cell(row= start_row + i, column= start_col + j, value=value)

suma_contratos = RESUMEN_EXP["No_De_contratos"].sum()
suma_venta = RESUMEN_EXP["Valor_de_venta_Precio_de_venta"].sum()
suma_cap = RESUMEN_EXP["Capital_insoluto_total"].sum()


fila_final = start_row + len(RESUMEN_EXP)


ws1.cell(3153, 5, suma_contratos)
ws1.cell(3153, 6, suma_venta)
ws1.cell(3153, 7, suma_cap)

ws1.delete_rows(fila_final, 3153 - fila_final)

ws2 = wb["Detalle"]

Detalle = Detalle.sort_values(by="Venta_real")

for i, row in enumerate(Detalle.values):
    for j, value in enumerate(row):
        ws2.cell(row=start_row + i, column= start_col + j, value=value)

ws2.cell(1108,5, suma_contratos)
ws2.cell(1108,6, suma_cap)

fila_final2 = start_row + len(Detalle)

ws2.delete_rows(fila_final2, 1108 - fila_final2)


wb.save(f"./{ruta_guardado}/Solicitud_Pagares_{fecha_str}.xlsx")
print("Se guardo archivo de venta de Luis.")

Se guardo archivo de venta de Luis.
